# Basics




In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.utils.data import clear_download_cache
clear_download_cache()

from spextra import Spextrum, spextra_database, SpecLibrary, FilterSystem

## Examining the database

In [ ]:
sdb = spextra_database

### Information of the database

In [ ]:
print(sdb)

In [ ]:
# Which templates are available?

print(SpecLibrary("sne"))

### Extinction curves and Filters

In [ ]:
print(sdb["extinction_curves"])
print(sdb["filter_systems"])

In [ ]:
print(FilterSystem("micado"))

## Retrieving the spectra

In [ ]:
sp1 = Spextrum("kc96/s0")
sp1.plot()

In [ ]:
# another spectrum

sp2 = Spextrum("agn/qso")
sp2.plot()

### Aritmetics

simple arithmetics are possible


In [ ]:
sp = sp1 + 3*sp2

sp.plot()

### Adding emission lines

It is possible to add emission lines, either individually or as a list. Parameters are center, flux and fwhm


In [ ]:
sp3 = sp1.add_emi_lines(center=4000,flux=4e-13, fwhm=5*u.AA)
sp3.plot(left=3500, right=4500)

### Redshifting spectra


In [ ]:

fig = plt.figure(figsize=(10,7))
sp4 = sp2.redshift(z=1)

wave = sp2.waveset
flux = sp2(wave, flux_unit="FLAM")

plt.plot(wave, flux)

plt.plot(sp4.waveset, 
         sp4(sp4.waveset, flux_unit="FLAM"))

plt.legend(['z=0', 'z=1'], loc='upper right')

### Or using velocity

In [ ]:
fig = plt.figure(figsize=(10,6))

sp1 = Spextrum("nebulae/orion")

vel = -1000 * u.km / u.s
sp2 = sp1.redshift(vel=vel)

plt.plot(sp1.waveset, 
         sp1(sp1.waveset, flux_unit="FLAM"))
plt.plot(sp2.waveset, 
         sp2(sp2.waveset, flux_unit="FLAM"))
plt.legend(['vel=0', 'vel=-1000 km/s'], loc='upper right')
plt.xlim(3000,5000)

### Flat spectrum in any photometric system

(aka reference spectrum if mag=0)

In [ ]:
sp_vega = Spextrum.flat_spectrum(10*u.mag)
sp_ab = Spextrum.flat_spectrum(10*u.ABmag)
sp_st = Spextrum.flat_spectrum(10*u.STmag)


fig = plt.figure(figsize=(10,7))
wave = sp_vega.waveset
plt.plot(wave, sp_vega(wave, flux_unit="FLAM"), label="Vega")
plt.plot(wave, sp_ab(wave, flux_unit="FLAM"), label="AB")
plt.plot(wave, sp_st(wave, flux_unit="FLAM"), label="ST")

plt.xlim(3000,1e4)
plt.ylim(0,0.2e-11)
plt.xlabel("wavelength")
plt.ylabel("flux (FLAM)")
plt.legend()

### Scaling to a magnitude

In [ ]:
sp1 = Spextrum("kc96/s0").scale_to_magnitude(amplitude=13 * u.ABmag, filter_curve="g")
sp2 = sp1.scale_to_magnitude(amplitude=15 * u.ABmag, filter_curve="g")

sig = plt.figure(figsize=(10,7))
plt.plot(sp1.waveset, 
         sp1(sp1.waveset, flux_unit="FLAM"))
plt.plot(sp2.waveset, 
         sp2(sp2.waveset, flux_unit="FLAM"))
plt.legend(['mag=13', 'mag=15'], loc='upper right')
plt.xlim(4000,7000)

### Obtaining magnitudes from spectra

In [ ]:
print("Magnitude spectra 1:", sp1.get_magnitude(filter_curve="g"), 
      sp1.get_magnitude(filter_curve="g", system_name="Vega"), "Vega")
print("Magnitude spectra 2:", sp2.get_magnitude(filter_curve="g"), 
      sp2.get_magnitude(filter_curve="g", system_name="Vega"), "Vega")

### Rebin spectra

a new wavelength array must be passed

In [ ]:
sp1 = Spextrum("agn/qso")
new_waves = np.linspace(np.min(sp1.waveset),
                        np.max(sp1.waveset),
                        100)
sp2 = sp1.rebin_spectra(new_waves=new_waves)

sig = plt.figure(figsize=(10,7))
plt.plot(sp1.waveset, 
         sp1(sp1.waveset, flux_unit="FLAM"))
plt.plot(sp2.waveset, 
         sp2(sp2.waveset, flux_unit="FLAM"))
plt.xlim(1000,4000)

### Smooth the spectral with a velocity kernel

In [ ]:
sp1 = Spextrum("nebulae/pn")

sigma = 500*(u.km / u.s)
sp2 = sp1.smooth(sigma=sigma)

fig = plt.figure(figsize=(10,7))
plt.plot(sp1.waveset, 
         sp1(sp1.waveset, flux_unit="FLAM"), label="original")
plt.plot(sp2.waveset, 
         sp2(sp2.waveset, flux_unit="FLAM"), label="broadened with 500 km/s")

plt.xlim(4800,5200)
plt.legend()

### Blackbody spectrum and extinction curves




In [ ]:
sp1 = Spextrum.black_body_spectrum(temperature=5500, 
                                   amplitude=10 * u.ABmag, 
                                   filter_curve="r")
sp2 = sp1.redden("gordon/smc_bar", Ebv=0.15)


fig = plt.figure(figsize=(10,7))
plt.plot(sp1.waveset, 
         sp1(sp1.waveset, flux_unit="FLAM"), label="original")
plt.plot(sp2.waveset, 
         sp2(sp2.waveset, flux_unit="FLAM"), label="attenuated")

plt.xlim(1800,15200)
plt.legend()


### Photons within a filter 

(or between wmin or wmax)

In [ ]:
n_photons = sp2.photons_in_range(area=2*u.m**2,
                                 filter_curve="V")

print(n_photons)